# Session 8 - Language modelling with RNNs (Text Generation)

In [1]:
# data processing tools
import string, os 
import pandas as pd
import numpy as np
np.random.seed(42)

# keras module for building LSTM 
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.utils as ku 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# surpress warnings
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-03-24 12:20:03.419601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Some helper functions

In [2]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower() # make all text lowercase, keep strings that arent punctuation
    txt = txt.encode("utf8").decode("ascii",'ignore') # utf8 encoding makes up for accents
    return txt 

def get_sequence_of_tokens(tokenizer, corpus):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences

def generate_padded_sequences(input_sequences):
    # get the length of the longest sequence
    max_sequence_len = max([len(x) for x in input_sequences])
    # make every sequence the length of the longest on
    input_sequences = np.array(pad_sequences(input_sequences, 
                                            maxlen=max_sequence_len, 
                                            padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, 
                            num_classes=total_words)
    return predictors, label, max_sequence_len

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 
                        10, 
                        input_length=input_len))
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1)) # remove 10% of the weight when the model is training
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    return model

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], 
                                    maxlen=max_sequence_len-1, 
                                    padding='pre')
        predicted = np.argmax(model.predict(token_list),
                                            axis=1)   
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## Load the data

In [3]:
data_dir = os.path.join("../in/news_data")

We're then going to load the data one at a time and append *only* the headlines to our list of data.

In [4]:
all_comments = []
for filename in os.listdir(data_dir):
    if 'Comments' in filename:
        comment_df = pd.read_csv(data_dir + "/" + filename)
        all_comments.extend(list(comment_df["commentBody"].values))

We then clean up a little bit and see how many data points we have.

In [5]:
all_comments = [c for c in all_comments if c != "Unknown"] # remove all unknown headlines
len(all_comments)

418481

In [20]:
# create sample from all_comments
import random
sample_comments = random.sample(all_comments, 100)
len(sample_comments)

100

We call out ```clean_text()``` function and then inspect the first 10 texts.

In [21]:
corpus = [clean_text(x) for x in sample_comments]
corpus[:10]

['the pension bomb is real for many categories of public retirees especially police and fire  your perception of the teacher glide path being stressful  actually thats the private sector as well  i greatly value all who perform honest work including teachers and public employees  but why should public employees of any category have it easier or better than the general working population that is footing the bill',
 'its beautiful ',
 'a signet is also called a sealbra hrefhttpswwwmerriamwebstercomdictionaryseal titlehttpswwwmerriamwebstercomdictionaryseal targetblankhttpswwwmerriamwebstercomdictionarysealabrbrb 1  a device with a cut or raised emblem symbol or word used especially to certify a signature or authenticate a document 2  a medallion or ring face bearing such a device incised so that it can be impressed on wax or moist clay',
 'trump also likes medals remember the purple heart he was given so he should probably award himself the presidential medal of freedom for his fantasy o

## Tokenize

We're then going to tokenize our data, using the ```Tokenizer()``` class from ```TensorFlow```, about which you can read more [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer).

We then use the ```get_sequence_of_tokens()``` function we defined above, which turns every text into a sequence of tokens based on the vocabulary from the tokenizer.

In [22]:
# this code creates an index of the words in the corpus
tokenizer = Tokenizer()
## tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 # +1 accounts for words that arent in the vocabulary. 

In [23]:
# take input and turn it into numerical output
inp_sequences = get_sequence_of_tokens(tokenizer, corpus)
inp_sequences[:10]
# the length of the input sequences are defined by how the words relate to each other

[[1, 708],
 [1, 708, 709],
 [1, 708, 709, 7],
 [1, 708, 709, 7, 390],
 [1, 708, 709, 7, 390, 11],
 [1, 708, 709, 7, 390, 11, 69],
 [1, 708, 709, 7, 390, 11, 69, 710],
 [1, 708, 709, 7, 390, 11, 69, 710, 5],
 [1, 708, 709, 7, 390, 11, 69, 710, 5, 83],
 [1, 708, 709, 7, 390, 11, 69, 710, 5, 83, 711]]

We then want to *pad* our input sequences to make them all the same length.

In [24]:
# padding makes different inputs the same length as the longest input by adding 0's where words are missing in inputs
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
# E.g. with "My cat" and "The big dog" a 0 would be added in the front like "0 My cat"

## Create model

We then use the ```create_model()``` function created above to initialize a model, telling the model the length of sequences and the total size of the vocabulary.

In [25]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 263, 10)           24090     
                                                                 
 lstm_2 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 2409)              243309    
                                                                 
Total params: 311,799
Trainable params: 311,799
Non-trainable params: 0
_________________________________________________________________


2023-03-24 12:22:46.441197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-24 12:22:46.443836: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-24 12:22:46.445864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model training is exactly the same as last week, but instead of document labels, we're fitting the model to predict next word.

*NB!* This will take some time to train! It took me 35 minutes on UCloud 32xCPU.

In [26]:
# creating history of the model
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

Epoch 1/100


2023-03-24 12:22:51.462663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-24 12:22:51.465472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-24 12:22:51.467542: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

53/53 [==============================] - 24s 382ms/step - loss: 7.2937
Epoch 2/100
53/53 [==============================] - 23s 434ms/step - loss: 6.7114
Epoch 3/100
53/53 [==============================] - 23s 426ms/step - loss: 6.6454
Epoch 4/100
53/53 [==============================] - 23s 425ms/step - loss: 6.6069
Epoch 5/100
53/53 [==============================] - 23s 431ms/step - loss: 6.5649
Epoch 6/100
53/53 [==============================] - 23s 424ms/step - loss: 6.5206
Epoch 7/100
53/53 [==============================] - 23s 425ms/step - loss: 6.4808
Epoch 8/100
53/53 [==============================] - 22s 422ms/step - loss: 6.4421
Epoch 9/100
53/53 [==============================] - 22s 421ms/step - loss: 6.4105
Epoch 10/100
53/53 [==============================] - 23s 429ms/step - loss: 6.3810
Epoch 11/100
53/53 [==============================] - 22s 423ms/step - loss: 6.3519
Epoch 12/100
53/53 [==============================] - 23s 428ms/step - loss: 6.3204
Epoch 13/100


When the model has trained, we can then use this to generate *new text*.

In [27]:
# print text based on the word/words and what the model has predicted would go along with it
print(generate_text("danish", 5, model, max_sequence_len)) # the 5 is the number of words we want to come after

2023-03-24 13:11:58.672912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-24 13:11:58.675595: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-24 13:11:58.677490: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 34ms/step
Danish The World Latestage Capitalism The


In [31]:
# saving
outpath = os.path.join("../out/rnn_model.joblib")
tf.keras.saving.save_model(model, outpath, overwrite=True, save_format=None)

NameError: name 'kwargs' is not defined

In [35]:
# importing
new_model = tf.keras.models.load_model("../out/rnn_model.joblib")

2023-03-24 13:19:41.153698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-03-24 13:19:41.153817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-03-24 13:19:41.153910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [37]:
print(generate_text("danish", 5, new_model, max_sequence_len))

1/1 [==============================] - 1s 589ms/step


2023-03-24 13:20:18.562771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-24 13:20:18.565668: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-24 13:20:18.567803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 48ms/step
Danish The World Latestage Capitalism The


In [ ]:
import joblib
from joblib import dump
# saving trained model
dump(history, "../out/rnn_model.joblib")

In [28]:
# load model
loaded_model = joblib.load("../out/rnn_model.joblib")
print(generate_text("danish", 5, model, max_sequence_len))

2023-03-22 14:26:42.620445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-03-22 14:26:42.623448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-03-22 14:26:42.625440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 21ms/step
Danish Inventor Is Found Guilty In


## Using pre-trained word embeddings

Instead of having the embedding layer as a trainable parameter, we can instead using a *pretrained word embedding* model like ```word2vec```.

In the following examples, we're using [GloVe embeddings](https://nlp.stanford.edu/projects/glove/). These are trained a little differently from ```word2vec``` but they behave in the same way.

We make use of already trained word embeddings, which are better than what we ever could train on a model ourselves.

In [42]:
# defining path
path_to_glove_file = os.path.join("../data/glove/")

# creating pre-trained word embedding index
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

KeyboardInterrupt: 

We can define some variables that we're going to use later.

With hits and misses, we're counting how many words in the corpus vocabulary have a corresponding GloVe embedding; misses are the words which appear in our vocabulary but which do not have a GloVe embedding.

In [ ]:
num_tokens = total_words
embedding_dim = 100
hits = 0
misses = 0

In [ ]:
# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer - notice that this is different
    model.add(Embedding(
            total_words,
            embedding_dim,
            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
            trainable=False,
            input_length=input_len)
    )
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(500))
    model.add(Dropout(0.2))
    
    # Add Output Layer
    model.add(Dense(total_words, 
                    activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                    optimizer='adam')
    
    return model

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
history = model.fit(predictors, 
                    label, 
                    epochs=100,
                    batch_size=128, 
                    verbose=1)

In [ ]:
print (generate_text("china", 30, model, max_sequence_len))